In [2]:
import os
import pandas as pd

In [9]:
def get_sample_id(sample_name):
    return int(sample_name.split('_')[3].split('-')[2])

def get_run_id(sample_name):
    return sample_name.split('_')[0]

def get_sample_name_prev(sample_name):
    
    if 'R1.fastq.gz' in sample_name:
        return sample_name.replace('_R1.fastq.gz', '')
    
    elif 'R2.fastq.gz' in sample_name:
        return sample_name.replace('_R2.fastq.gz', '')
    
    else:
        print(sample_name, 'Not a paired end fastq reads!')

In [19]:
samples_fluente = pd.read_csv('./data/samples_name_fluente.tsv', sep='\t', names = ['sample_name'])
samples_fluente['ID'] = samples_fluente['sample_name'].apply(lambda x: get_sample_id(x))
# samples_fluente['run_id'] = samples_fluente['sample_name'].apply(lambda x: get_run_id(x))

samples_fluente

sample_name     ID
0    run221223_UVZ_BA_22-vsp-41416_R1.fastq.gz  41416
1    run221223_UVZ_BA_22-vsp-41416_R2.fastq.gz  41416
2    run221223_UVZ_BA_22-vsp-41636_R1.fastq.gz  41636
3    run221223_UVZ_BA_22-vsp-41636_R2.fastq.gz  41636
4    run221223_UVZ_BA_22-vsp-41637_R1.fastq.gz  41637
..                                         ...    ...
941   run230710_UVZ_BA_23-vsp-5055_R2.fastq.gz   5055
942   run230710_UVZ_BA_23-vsp-5433_R1.fastq.gz   5433
943   run230710_UVZ_BA_23-vsp-5433_R2.fastq.gz   5433
944   run230710_UVZ_BA_23-vsp-5517_R1.fastq.gz   5517
945   run230710_UVZ_BA_23-vsp-5517_R2.fastq.gz   5517

[946 rows x 2 columns]

In [5]:
samples = pd.read_csv("./data/samples_name.tsv", sep='\t', names=['sample_name'])
samples['ID'] = samples['sample_name'].apply(lambda x: get_sample_id(x))
samples['run_id'] = samples['sample_name'].apply(lambda x: get_run_id(x))

samples

sample_name     ID     run_id
0    run221223_UVZ_BA_22-vsp-41416_R1.fastq.gz  41416  run221223
1    run221223_UVZ_BA_22-vsp-41416_R2.fastq.gz  41416  run221223
2    run221223_UVZ_BA_22-vsp-41636_R1.fastq.gz  41636  run221223
3    run221223_UVZ_BA_22-vsp-41636_R2.fastq.gz  41636  run221223
4    run221223_UVZ_BA_22-vsp-41637_R1.fastq.gz  41637  run221223
..                                         ...    ...        ...
921   run230522_UVZ_BA_23-vsp-4459_R2.fastq.gz   4459  run230522
922   run230522_UVZ_BA_23-vsp-4595_R1.fastq.gz   4595  run230522
923   run230522_UVZ_BA_23-vsp-4595_R2.fastq.gz   4595  run230522
924   run230522_UVZ_BA_23-vsp-4690_R1.fastq.gz   4690  run230522
925   run230522_UVZ_BA_23-vsp-4690_R2.fastq.gz   4690  run230522

[926 rows x 3 columns]

In [6]:
samples['ID'].drop_duplicates()

0      41416
2      41636
4      41637
6      41638
8      41724
       ...  
916     4457
918     4458
920     4459
922     4595
924     4690
Name: ID, Length: 463, dtype: int64

In [7]:
# merge all results form google sheets into one pandas dataframe

column_names = ['sample_name', 'ID', 'run_id', 'type']
samples_results_df = pd.DataFrame(columns=column_names)

results_file = ['10.7', '2.5', '20.3', '22.5', '27.03', '27.2', '3.4']

for res_file in results_file: # itterate over tsv files
    res_file_df = pd.read_csv('./data/vysledky/Respiračné_výsledky - CH-' + res_file + '.2023.tsv', sep='\t')
    res_file_df = res_file_df[['ID', 'type']] # 'type' predikovany vysledok virusu (najpravdepodobnejsi vysledok)

    # list of samples from directory (reads/original)
    for _, sample in samples.iterrows():
        
        # list of results
        for _, res in res_file_df.iterrows():
            if type(res['ID'] ) is not int and res['ID'] != 'NK':
                res_id = res['ID'].split('/')[1]
            else:
                res_id = res['ID']

            # concatenate
            if int(sample['ID']) == res_id:
                sample_df = pd.DataFrame([[sample['sample_name'], sample['ID'], sample['run_id'], res['type']]] , columns=column_names)

                samples_results_df = pd.concat([samples_results_df, sample_df], ignore_index=True)


In [8]:
samples_results_df.groupby(['type']).count()

sample_name   ID  run_id
type                                
h1n1                 10   10      10
h3n2                  2    2       2
rsv_b                 2    2       2
sars_cov_2           12   12      12
victoria            104  104     104

In [11]:
samples_results_df.sort_values('type').to_csv('./data/vysledky/samples_results.tsv', sep='\t')

In [4]:
pd.read_csv('./data/samples_results.tsv', sep='\t')

sample_name    ID                    type  \
0    run230710_UVZ_BA_23-vsp-4691_R1.fastq.gz  4691  Human coronavirus HKU1   
1    run230710_UVZ_BA_23-vsp-4691_R2.fastq.gz  4691  Human coronavirus HKU1   
2    run230710_UVZ_BA_23-vsp-4692_R1.fastq.gz  4692                victoria   
3    run230710_UVZ_BA_23-vsp-4692_R2.fastq.gz  4692                victoria   
4    run230710_UVZ_BA_23-vsp-4946_R1.fastq.gz  4946                victoria   
..                                        ...   ...                     ...   
265  run230403_UVZ_BA_23-vsp-3524_R2.fastq.gz  3524              sars_cov_2   
266  run230403_UVZ_BA_23-vsp-3525_R1.fastq.gz  3525              sars_cov_2   
267  run230403_UVZ_BA_23-vsp-3525_R2.fastq.gz  3525              sars_cov_2   
268  run230403_UVZ_BA_23-vsp-3526_R1.fastq.gz  3526                     NaN   
269  run230403_UVZ_BA_23-vsp-3526_R2.fastq.gz  3526                     NaN   

     coverage  totalSubstitutions  
0         NaN                 NaN  
1         NaN                 NaN  
2    0.932626                48.0  
3    0.932626                48.0  
4    0.932626                47.0  
..        ...                 ...  
265  0.999398                99.0  
266  0.999498               102.0  
267  0.999498               102.0  
268       NaN                 NaN  
269       NaN                 NaN  

[270 rows x 5 columns]

In [114]:
# get samples ID accorsing to sampleresult ('type') 
list(samples_results_df[samples_results_df['type'] == 'victoria'].sort_values('ID').drop_duplicates('ID')['ID'])


[3270,
 3328,
 3336,
 3338,
 3339,
 3340,
 3342,
 3343,
 3344,
 3345,
 3420,
 3456,
 3457,
 3481,
 3482,
 3517,
 3518,
 3519,
 3520,
 3521,
 3944,
 3945,
 3946,
 3947,
 3948,
 3949,
 3950,
 4072,
 4084,
 4127,
 4128,
 4129,
 4130,
 4131,
 4132,
 4133,
 4134,
 4135,
 4136,
 4137,
 4242,
 4243,
 4244,
 4245,
 4317,
 4318,
 4319,
 4457,
 4458,
 4459,
 4595,
 4690]

In [47]:
# count how many samples and results are in google sheets

result_count = 0
for res_file in results_file: # itterate over tsv files
    res_file_df = pd.read_csv('./data/vysledky/Respiračné_výsledky - CH-' + res_file + '.2023.tsv', sep='\t')
    result_count += res_file_df.shape[0]

result_count

329

In [10]:
# pridanie stlpca obs nazov vzorky(bez _R1/2.fastq.gz)
samples_results_df['sample_name_prev'] = samples_results_df['sample_name'].apply(lambda x: get_sample_name_prev(x))
samples_results_df # malo by ich byt 464

sample_name    ID                    type  \
0    run230710_UVZ_BA_23-vsp-4691_R1.fastq.gz  4691  Human coronavirus HKU1   
1    run230710_UVZ_BA_23-vsp-4691_R2.fastq.gz  4691  Human coronavirus HKU1   
2    run230710_UVZ_BA_23-vsp-4692_R1.fastq.gz  4692                victoria   
3    run230710_UVZ_BA_23-vsp-4692_R2.fastq.gz  4692                victoria   
4    run230710_UVZ_BA_23-vsp-4946_R1.fastq.gz  4946                victoria   
..                                        ...   ...                     ...   
265  run230403_UVZ_BA_23-vsp-3524_R2.fastq.gz  3524              sars_cov_2   
266  run230403_UVZ_BA_23-vsp-3525_R1.fastq.gz  3525              sars_cov_2   
267  run230403_UVZ_BA_23-vsp-3525_R2.fastq.gz  3525              sars_cov_2   
268  run230403_UVZ_BA_23-vsp-3526_R1.fastq.gz  3526                     NaN   
269  run230403_UVZ_BA_23-vsp-3526_R2.fastq.gz  3526                     NaN   

     coverage  totalSubstitutions              sample_name_prev  
0         NaN                 NaN  run230710_UVZ_BA_23-vsp-4691  
1         NaN                 NaN  run230710_UVZ_BA_23-vsp-4691  
2    0.932626                48.0  run230710_UVZ_BA_23-vsp-4692  
3    0.932626                48.0  run230710_UVZ_BA_23-vsp-4692  
4    0.932626                47.0  run230710_UVZ_BA_23-vsp-4946  
..        ...                 ...                           ...  
265  0.999398                99.0  run230403_UVZ_BA_23-vsp-3524  
266  0.999498               102.0  run230403_UVZ_BA_23-vsp-3525  
267  0.999498               102.0  run230403_UVZ_BA_23-vsp-3525  
268       NaN                 NaN  run230403_UVZ_BA_23-vsp-3526  
269       NaN                 NaN  run230403_UVZ_BA_23-vsp-3526  

[270 rows x 6 columns]

In [132]:
samples_results_df = samples_results_df.drop_duplicates('ID')
samples_results_df_h3n2 = samples_results_df[samples_results_df['type'] == 'h3n2']
samples_results_df_h3n2

sample_name    ID     run_id  type  \
0  run230502_UVZ_BA_23-vsp-3943_R1.fastq.gz  3943  run230502  h3n2   

               sample_name_prev  
0  run230502_UVZ_BA_23-vsp-3943

In [85]:
list(samples['run_id'].drop_duplicates())

['run221223',
 'run230131',
 'run230213',
 'run230320',
 'run230403',
 'run230418',
 'run230502',
 'run230522']

In [ ]:
# vytvorenie nazvu suborov podla zmeny parametro (generuj nazov podla parametrov)

In [120]:
runs = list(samples['run_id'].drop_duplicates())
trimmers = ['trimmom', 'cudapt']
trimmed_qualities = [10, 15, 20, 25, 30]
headcrops = [0, 5, 10, 15]
minlens = [35, 45, 55, 65]
# set temp = True !!!

In [11]:
samples_results_df = pd.read_csv('./data/samples_results.tsv', sep='\t')
samples_results_df['sample_name_prev'] = samples_results_df['sample_name'].apply(lambda x: get_sample_name_prev(x))
samples_results_df 

sample_name    ID                    type  \
0    run230710_UVZ_BA_23-vsp-4691_R1.fastq.gz  4691  Human coronavirus HKU1   
1    run230710_UVZ_BA_23-vsp-4691_R2.fastq.gz  4691  Human coronavirus HKU1   
2    run230710_UVZ_BA_23-vsp-4692_R1.fastq.gz  4692                victoria   
3    run230710_UVZ_BA_23-vsp-4692_R2.fastq.gz  4692                victoria   
4    run230710_UVZ_BA_23-vsp-4946_R1.fastq.gz  4946                victoria   
..                                        ...   ...                     ...   
265  run230403_UVZ_BA_23-vsp-3524_R2.fastq.gz  3524              sars_cov_2   
266  run230403_UVZ_BA_23-vsp-3525_R1.fastq.gz  3525              sars_cov_2   
267  run230403_UVZ_BA_23-vsp-3525_R2.fastq.gz  3525              sars_cov_2   
268  run230403_UVZ_BA_23-vsp-3526_R1.fastq.gz  3526                     NaN   
269  run230403_UVZ_BA_23-vsp-3526_R2.fastq.gz  3526                     NaN   

     coverage  totalSubstitutions              sample_name_prev  
0         NaN                 NaN  run230710_UVZ_BA_23-vsp-4691  
1         NaN                 NaN  run230710_UVZ_BA_23-vsp-4691  
2    0.932626                48.0  run230710_UVZ_BA_23-vsp-4692  
3    0.932626                48.0  run230710_UVZ_BA_23-vsp-4692  
4    0.932626                47.0  run230710_UVZ_BA_23-vsp-4946  
..        ...                 ...                           ...  
265  0.999398                99.0  run230403_UVZ_BA_23-vsp-3524  
266  0.999498               102.0  run230403_UVZ_BA_23-vsp-3525  
267  0.999498               102.0  run230403_UVZ_BA_23-vsp-3525  
268       NaN                 NaN  run230403_UVZ_BA_23-vsp-3526  
269       NaN                 NaN  run230403_UVZ_BA_23-vsp-3526  

[270 rows x 6 columns]

In [12]:
trimmers = ['trimmom', 'cudapt']
trimmed_qualities = [10, 15, 20, 25, 30]
headcrops = [0, 5, 10, 15]
minlens = [35, 45, 55, 65]
# set temp = True !!!

# vytvorenie zoznamu (v podobe dataframemu) nazvov suborov

sample_file_names_opt = pd.DataFrame(columns=['sample_name'])

# iba nazov suboru bez pairend citani (koncoviek R1.gz/R2.gz)
for sample_name_prev in samples_results_df['sample_name_prev']:
    for trimmer in trimmers:
        for trimmed_quality in trimmed_qualities:
            for headcrop in headcrops:
                for minlen in minlens:
                    
                    # paired end reads
                    for i in [1, 2]:
                        # report_dir aj sample_name

                        sample_name = sample_name_prev + '_' + trimmer + '_tripq' + str(trimmed_quality) + '_headcrop' + str(headcrop) + '_minlen' + str(minlen) + '_R' + str(i) + '.fastq.gz'
                        
                        sample_file_names_opt = pd.concat([sample_file_names_opt, pd.DataFrame([[sample_name]], columns=['sample_name'])], ignore_index=True)

sample_file_names_opt

sample_name
0      run230710_UVZ_BA_23-vsp-4691_trimmom_tripq10_h...
1      run230710_UVZ_BA_23-vsp-4691_trimmom_tripq10_h...
2      run230710_UVZ_BA_23-vsp-4691_trimmom_tripq10_h...
3      run230710_UVZ_BA_23-vsp-4691_trimmom_tripq10_h...
4      run230710_UVZ_BA_23-vsp-4691_trimmom_tripq10_h...
...                                                  ...
86395  run230403_UVZ_BA_23-vsp-3526_cudapt_tripq30_he...
86396  run230403_UVZ_BA_23-vsp-3526_cudapt_tripq30_he...
86397  run230403_UVZ_BA_23-vsp-3526_cudapt_tripq30_he...
86398  run230403_UVZ_BA_23-vsp-3526_cudapt_tripq30_he...
86399  run230403_UVZ_BA_23-vsp-3526_cudapt_tripq30_he...

[86400 rows x 1 columns]

In [137]:
print(sample_file_names_opt['sample_name'][0])
sample_file_names_opt['sample_name'][1]

run230502_UVZ_BA_23-vsp-3943_trimmom_tripq10_headcrop0_minlen35_R1.fastq.gz


'run230502_UVZ_BA_23-vsp-3943_trimmom_tripq10_headcrop0_minlen35_R2.fastq.gz'

In [3]:
# vytvor hardlinky pre zmenene parametre
import os
import pandas as pd
# cez vsetky vzorky
for sample_file_name in sample_file_names_opt.iterrows():
    os.link(sample_name, sample_file_name)
    os.symlink()

In [ ]:
# vyber iba vzorky pre chripky (ku ktorym mame konsenzus)

In [ ]:
# Vytvor config so zmenenou Baseline

In [27]:
import yaml

# Load the YAML file
with open('./scripts/variant_call/variable_test.yaml', 'r') as file:
    data = yaml.safe_load(file)

    print(data)
    print(data['samples'][0]['reference'])


{'sequencing': 'paired_end', 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}], 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/', 'threads': 16, 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic', 'temporary': False, 'crop': 500, 'quality': 20, 'headcrop': 0, 'minlen': 45}}}}
h1n1_2019


In [18]:
import pprint
pprint.pprint(data)

{'reads': {'preprocess': {'trimmed': {'crop': 500,
                                      'headcrop': 0,
                                      'method': 'trimmomatic',
                                      'minlen': 45,
                                      'quality': 20,
                                      'temporary': False}}},
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/',
 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}],
 'sequencing': 'paired_end',
 'threads': 16}


In [28]:
data

{'sequencing': 'paired_end',
 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}],
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/',
 'threads': 16,
 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic',
    'temporary': False,
    'crop': 500,
    'quality': 20,
    'headcrop': 0,
    'minlen': 45}}}}

In [21]:
data['samples'][0]['reference'] = 'aaa'
data['sequencing'] = 'xxxx'
data

{'sequencing': 'xxxx',
 'samples': [{'name': 'run230710.*', 'reference': 'aaa'}],
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/',
 'threads': 16,
 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic',
    'temporary': False,
    'crop': 500,
    'quality': 20,
    'headcrop': 0,
    'minlen': 45}}}}

In [30]:
data

{'sequencing': 'paired_end',
 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}],
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/',
 'threads': 16,
 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic',
    'temporary': False,
    'crop': 500,
    'quality': 20,
    'headcrop': 0,
    'minlen': 45}}}}

In [33]:
with open('./scripts/variant_call/var_test1.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)



In [16]:
# d = dict()
d = {'sequencing': 'paired_end','samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}], }
pprint.pprint(d)
d['samples'][0]['reference'] = 'aaa'
d

{'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}],
 'sequencing': 'paired_end'}


{'sequencing': 'paired_end',
 'samples': [{'name': 'run230710.*', 'reference': 'aaa'}]}

In [41]:
print(yaml.dump(data, default_flow_style=True))

{reads: {preprocess: {trimmed: {crop: 500, headcrop: 0, method: trimmomatic, minlen: 45,
        quality: 20, temporary: false}}}, report_dir: report/public/17-trimmomatic-ivar-vsp-run230710/,
  samples: [{name: run230710.*, reference: h1n1_2019}], sequencing: paired_end, threads: 16}



In [57]:
from collections import OrderedDict

data_ordered = OrderedDict({'sequencing': 'paired_end', 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}], 
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/', 'threads': 16, 
 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic', 'temporary': False, 'crop': 500, 'quality': 20, 'headcrop': 0, 'minlen': 45}}}}
)
print(data_ordered)
data_dict = dict(data_ordered)
data_dict


OrderedDict([('sequencing', 'paired_end'), ('samples', [{'name': 'run230710.*', 'reference': 'h1n1_2019'}]), ('report_dir', 'report/public/17-trimmomatic-ivar-vsp-run230710/'), ('threads', 16), ('reads', {'preprocess': {'trimmed': {'method': 'trimmomatic', 'temporary': False, 'crop': 500, 'quality': 20, 'headcrop': 0, 'minlen': 45}}})])


{'sequencing': 'paired_end',
 'samples': [{'name': 'run230710.*', 'reference': 'h1n1_2019'}],
 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/',
 'threads': 16,
 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic',
    'temporary': False,
    'crop': 500,
    'quality': 20,
    'headcrop': 0,
    'minlen': 45}}}}

In [59]:
with open('./scripts/variant_call/var_test_ordered_dict.yaml', 'w') as file:
    yaml.dump(data_dict, file, default_flow_style=True)

In [60]:
import yaml
from collections import OrderedDict

# Create an OrderedDict (to preserve the order of items)
data = OrderedDict({
    'first_name': 'John',
    'last_name': 'Doe',
    'age': 30
})

# Convert the OrderedDict to a regular dictionary
data_dict = dict(data)

# Save to YAML file in a simple format
with open('output.yaml', 'w') as file:
    yaml.dump(data_dict, file, default_flow_style=True)

print('Dictionary saved to output.yaml')


Dictionary saved to output.yaml


In [ ]:
# ruamel

In [11]:
# Access the variable
variable_value = data['samples']
print(f"Current value: {variable_value}")

# Modify the variable
data['samples'][0]['name'] = 'new_sample_name'
# data['samples'][0]['reference'] = 'new_reference'

# Save the updated YAML file
with open('./scripts/variant_call/variable_test_out.yaml', 'w') as file:
    yaml.dump(data, file)


Current value: [{'name': 'run230710.*', 'reference': 'h1n1_2019'}]


In [12]:
with open('./scripts/variant_call/variable_test_out.yaml', 'r') as file:
    data = yaml.safe_load(file)

    print(data)


{'reads': {'preprocess': {'trimmed': {'crop': 500, 'headcrop': 0, 'method': 'trimmomatic', 'minlen': 45, 'quality': 20, 'temporary': False}}}, 'report_dir': 'report/public/17-trimmomatic-ivar-vsp-run230710/', 'samples': [{'name': 'new_sample_name', 'reference': 'h1n1_2019'}], 'sequencing': 'paired_end', 'threads': 16}


In [191]:
import sys
from ruamel.yaml import RoundTripLoader, RoundTripDumper, load, dump

# read baseline config file, in which will be later set parameters to other values 

text_file = open("./scripts/variant_call/ivar.yaml", "r")

#read whole file to a string
data = text_file.read()
#close file
text_file.close()

# set type: ruamel.yaml.comments.CommentedMap aby sa dalo prepisovat na spravnom mieste
conf_file_baseline = load(data, Loader=RoundTripLoader)
dump(conf_file_baseline, sys.stdout, Dumper=RoundTripDumper)

conf_file_baseline

sequencing: paired_end
samples:                             # List of sample categories to be analysed
#   - name: run230710.*               # Regex expression of sample names to be analysed (reads/original/example.*_R1.fastq.gz)
#      reference: h1n1_2019                 # Reference genome for reads in the category (reference/mhv/mhv.fa)
- name: run230502_UVZ_BA_23-vsp-3943.*
  reference: h3n2_2021
report_dir: report/public/h3n2_baseline_3943/
threads: 16
reads:                              # Prepare reads and quality reports for downstream analysis
  preprocess:                       # Pre-process of reads, eliminate sequencing artifacts, contamination ...
    trimmed:                        # Remove low quality parts of reads
      method: trimmomatic
      temporary: false
      crop: 500
      quality: 20
      headcrop: 0
      minlen: 45
    decontaminated:                 # Eliminate fragments from known artificial source, e.g. contamination by human
      method: bowtie2
    

{'sequencing': 'paired_end', 'samples': [{'name': 'run230502_UVZ_BA_23-vsp-3943.*', 'reference': 'h3n2_2021'}], 'report_dir': 'report/public/h3n2_baseline_3943/', 'threads': 16, 'reads': {'preprocess': {'trimmed': {'method': 'trimmomatic', 'temporary': False, 'crop': 500, 'quality': 20, 'headcrop': 0, 'minlen': 45}, 'decontaminated': {'method': 'bowtie2', 'temporary': True, 'references': ['grch38_decoy'], 'keep': False}}, 'report': {'quality_report': {'method': 'fastqc', 'read_types': ['original', 'trimmed', 'decontaminated']}}}, 'mapping': {'mapper': {'method': 'bwa', 'temporary': True}, 'index': {'method': 'samtools'}, 'postprocess': {'sorted': {'method': 'samtools', 'temporary': True}, 'deduplicated': {'method': 'picard'}}}, 'variant': {'caller': {'method': 'freebayes', 'min_mapping_quality': 15, 'min_base_quality': 20, 'min_alternate_count': 5, 'min_alternate_fraction': 0.5, 'min_coverage': 10, 'all_alleles': False, 'gvcf': False, 'ploidy': 2}}, 'consensus': {'fasta': {'method': 'i

In [210]:
runs = list(samples['run_id'].drop_duplicates())
virus_reference = 'h3n2_2021'
trimmers = ['trimmomatic', 'cudapt']
trimmed_qualities = [10, 15] #20, 25, 30]
headcrops = [0, 5, 10, 15]
minlens = [35, 45, 55, 65]
# set temp = True !!!


# iba nazov suboru bez pairend citani (koncoviek R1.gz/R2.gz)
for sample_name_prev in samples_results_df_h3n2['sample_name_prev']:
    for trimmer in trimmers:
        for trimmed_quality in trimmed_qualities:
            # for headcrop in headcrops:
                # for minlen in minlens:
                    
                    # paired end reads R1.gz and R2.gz
                    # for i in [1, 2]:
                        # report_dir aj sample_name

            sample_name = sample_name_prev + '_' + trimmer + '_tripq' + str(trimmed_quality) + '_headcrop' + str(headcrop) + '_minlen' + str(minlen) # + '_R' + str(i) + '.fastq.gz'
            
            # copy of baseline file
            conf_file = conf_file_baseline

            # change parameters
            conf_file['samples'][0]['name'] = sample_name
            conf_file['samples'][0]['reference'] = virus_reference

            conf_file['report_dir'] = 'report/public/' + sample_name
            conf_file['reads']['preprocess']['trimmed']['method'] = trimmer
            conf_file['reads']['preprocess']['trimmed']['quality'] = trimmed_quality

            # save to new yaml config file
            file_to_save = './scripts/variant_call/' + sample_name + '_ivar.yaml'
            text_file = open(file_to_save, "w")
            text_file.write(str(dump(conf_file, Dumper=RoundTripDumper)))
            text_file.close()

In [209]:
conf_file['reads']['preprocess']['trimmed']['method'] 

'trimmomatic'

In [168]:
data['samples'][0]['name'] = 'sample_name_xxx'

In [172]:
from ruamel.yaml import YAML
with open('./scripts/variant_call/ivar_change.yaml', 'w') as f:
    yaml.dump(data, f)

In [176]:
dump(data, sys.stdout, Dumper=RoundTripDumper)

sequencing: paired_end
samples:                             # List of sample categories to be analysed
#   - name: run230710.*               # Regex expression of sample names to be analysed (reads/original/example.*_R1.fastq.gz)
#      reference: h1n1_2019                 # Reference genome for reads in the category (reference/mhv/mhv.fa)
- name: sample_name_xxx
  reference: h3n2_2021
report_dir: report/public/h3n2_baseline_3943/
threads: 16
reads:                              # Prepare reads and quality reports for downstream analysis
  preprocess:                       # Pre-process of reads, eliminate sequencing artifacts, contamination ...
    trimmed:                        # Remove low quality parts of reads
      method: trimmomatic
      temporary: false
      crop: 500
      quality: 20
      headcrop: 0
      minlen: 45
    decontaminated:                 # Eliminate fragments from known artificial source, e.g. contamination by human
      method: bowtie2
      temporary: tr

In [185]:
# zmenit parametre a ulozit do yaml suboru vo formate ako povodne
data['reads']['preprocess']['trimmed']['method'] = 'cudadapt'
text_file = open("./scripts/variant_call/ivar_change.yaml", "w")

text_file.write(str(dump(data, Dumper=RoundTripDumper)))

text_file.close()


In [184]:
data['reads']['preprocess']['trimmed']['method'] = 'cudadapt'
data['reads']['preprocess']['trimmed']['method']

'cudadapt'

In [ ]:
# from create_configs.py

In [3]:
# je mozne pridavat do configu novy parameter, alebo mozno menit len existujuci?

import sys
from ruamel.yaml import RoundTripLoader, RoundTripDumper, load, dump

samples_names = {'h1n1': 'run230320_UVZ_BA_23-vsp-1337',
                 'h3n2': 'run230320_UVZ_BA_23-vsp-2129',
                 'rsv_a': 'run230320_UVZ_BA_23-vsp-1658',
                 'rsv_b': 'run230320_UVZ_BA_23-vsp-902',
                 'sars_cov_2_122': 'uvzsr-BA_24_00001-G12-BA_24_00000122',
                 'sars_cov_2_139': 'uvzsr-BA_24_00002-A09-BA_24_00000139',
                 'sars_cov_2_4964': 'run230710_UVZ_BA_23-vsp-4964'
                 }

virus_reference = {'h1n1': 'h1n1_2019',
                   'h3n2': 'h3n2_2021',
                   'rsv_a': 'rsv_a_2017',
                   'rsv_b': 'rsv_b_2019',
                   'sars_cov_2_122': 'sars_cov_2',
                   'sars_cov_2_139': 'sars_cov_2',
                   'sars_cov_2_4964': 'sars_cov_2'
                   }

variant_callers = ['bcftools', 'freebayes', 'ivar']

trimmers = ['trimmomatic', 'cutadapt']


for variant_caller in variant_callers:
# variant_caller = 'ivar'

    for trimmer in trimmers:
    
        # iba nazov suboru bez pairend citani (koncoviek R1.gz/R2.gz)
        for virus in samples_names.keys():
        
            sample_name_prev = samples_names[virus]
    
            # read baseline config file, in which will be later set parameters to other values
            config_path_name = "./scripts/variant_call/" + variant_caller + "_config_no_trimm_set.yaml"
            text_file = open(config_path_name, "r")
            #read whole file to a string
            data = text_file.read()
            #close file
            text_file.close()
            conf_file_baseline = load(data, Loader=RoundTripLoader)
            dump(conf_file_baseline, sys.stdout, Dumper=RoundTripDumper)
    
            
            sample_name = sample_name_prev + '_' + trimmer + '_' + variant_caller
                    
            # copy of baseline file
            conf_file = conf_file_baseline
            # change parameters
            conf_file['samples'][0]['name'] = sample_name
            conf_file['samples'][0]['reference'] = virus_reference[virus]
            conf_file['report_dir'] = 'report/public/' + sample_name + '/'
            conf_file['reads']['preprocess']['trimmed']['method'] = trimmer
            
            if trimmer == 'trimmomatic':
                conf_file['reads']['preprocess']['trimmed']['crop'] = 500
                conf_file['reads']['preprocess']['trimmed']['quality'] = 10
                conf_file['reads']['preprocess']['trimmed']['headcrop'] = 0
                conf_file['reads']['preprocess']['trimmed']['minlen'] = 35
            else:
                conf_file['reads']['preprocess']['trimmed']['minimum_length'] = 35
                q_cutoff = str(20) + ',' + str(20)
                conf_file['reads']['preprocess']['trimmed']['quality_cutoff'] = f'{(q_cutoff)}' # '"{}"'.format(q_cutoff)
                conf_file['reads']['preprocess']['trimmed']['length'] = 500
                conf_file['reads']['preprocess']['trimmed']['head_cut'] = 0
                conf_file['reads']['preprocess']['trimmed']['tail_cut'] = 0
            # save to new yaml config file
            file_to_save = './scripts/variant_call/final_comparison_configs/' + sample_name + '.yaml'
            text_file = open(file_to_save, "w")
            text_file.write(str(dump(conf_file, Dumper=RoundTripDumper)))
            text_file.close()
    

sequencing: paired_end
samples:                             # List of sample categories to be analysed                 
- name: run230502_UVZ_BA_23-vsp-3943.*
  reference: h3n2_2021
report_dir: report/public/h3n2_baseline_3943/
threads: 2
reads:                              # Prepare reads and quality reports for downstream analysis
  preprocess:                       # Pre-process of reads, eliminate sequencing artifacts, contamination ...
    trimmed:                        # Remove low quality parts of reads
      method: trimmomatic
      temporary: true
    decontaminated:                 # Eliminate fragments from known artificial source, e.g. contamination by human
      method: bowtie2
      temporary: true
      references:                   # List of reference genomes
      - grch38_decoy
      keep: false
mapping:                            # Find the most similar genomic region to reads in reference (mapping process)
  mapper:                           # Method for mapping


sequencing: paired_end
samples:                             # List of sample categories to be analysed                 
- name: run230502_UVZ_BA_23-vsp-3943.*
  reference: h3n2_2021
report_dir: report/public/h3n2_baseline_3943/
threads: 2
reads:                              # Prepare reads and quality reports for downstream analysis
  preprocess:                       # Pre-process of reads, eliminate sequencing artifacts, contamination ...
    trimmed:                        # Remove low quality parts of reads
      method: trimmomatic
      temporary: true
    decontaminated:                 # Eliminate fragments from known artificial source, e.g. contamination by human
      method: bowtie2
      temporary: true
      references:                   # List of reference genomes
      - grch38_decoy
      keep: false
mapping:                            # Find the most similar genomic region to reads in reference (mapping process)
  mapper:                           # Method for mapping
